In [96]:
import pandas
df = pandas.read_csv('../data/chart.csv')
# df.head(10)
df.columns=['time','a','b','c','production','consumption']
df=df.drop(['a', 'b', 'c'], axis=1)
df['time'] = pandas.to_datetime(df['time'], format='%d.%m.%Y %H:%M')
df=df[(df.consumption!=0) | (df.production!=0)]



(df[df.production!=0]).head(20)
# df.head(20)

# calculate difference between two rows

In [97]:
c=[]
prev=df.iloc[0]
for i, row in df.iterrows():
    c.append(row['consumption']-prev['consumption'])
    prev=row
df['consumption_d']=c

# ignore rows with difference <50 and consumption <250

In [98]:
df=df[(df.consumption_d<-50) | (df.consumption_d>50) | (df.consumption>250)]

# calculate wh between up and down

In [128]:
c=[]
prev=True
prevI=0
i=0
for index, row in df.iterrows():
    if row['consumption_d']>0 and not prev:
        count=0
        for x in range(prevI,i):
            verbrauch=abs(df.iloc[x+1]['consumption']-df.iloc[x]['consumption'])/2+df.iloc[x]['consumption']
            count+=verbrauch*(df.iloc[x+1]['time']-df.iloc[x]['time']).seconds/3600
        c.append(count)
        prevI=i
    elif row['consumption_d']<0 and prev:
        count=0
        for x in range(prevI,i):
            verbrauch=abs(df.iloc[x+1]['consumption']-df.iloc[x]['consumption'])/2+df.iloc[x]['consumption']
            count+=verbrauch*(df.iloc[x+1]['time']-df.iloc[x]['time']).seconds/3600
        c.append(count)
        prevI=i
    elif i==0:
        pass
    else:
        c.append(0)
    prev=row['consumption_d']>0
    i=i+1
#     if index==59:
#         break;
# print(c)
c.append(0)
df['wh']=c

In [131]:
(df[df.wh>100]).head(20)
(df[df.production!=0]).head(20)
# df.head(11)


,time,production,consumption,consumption_d,wh
519,2020-01-01 08:39:00,489,441.3391,242.4283,548.469483
529,2020-01-01 08:49:00,961,178.1862,-221.1847,112.851600
574,2020-01-01 09:34:00,2737,4649.2520,121.9747,1146.144383
584,2020-01-01 09:44:00,3098,1788.4115,1377.5770,185.092912
594,2020-01-01 09:54:00,3365,950.8220,27.8174,112.757333
609,2020-01-01 10:09:00,3700,186.1504,-159.7121,127.758583
649,2020-01-01 10:49:00,4646,2349.7048,2178.2502,305.115129
669,2020-01-01 11:09:00,5003,56.7349,-217.0737,532.910350
729,2020-01-01 12:09:00,5532,552.4082,221.9189,117.632979
764,2020-01-01 12:44:00,5417,85.9702,-212.2232,145.332746
